<a href="https://colab.research.google.com/github/ysuter/FHNW-BAI-ComputerVision/blob/main/W02/hough_ransac_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔍 Geometrische Formerkennung: Hough-Transform vs. RANSAC

**Lernziele:**
- ✅ Hough-Transform für Linien und Kreise verstehen
- ✅ RANSAC-Algorithmus verstehen und anwenden
- ✅ Beide Methoden vergleichen
- ✅ Parameter-Tuning lernen

---

## 📚 Theoretischer Hintergrund

### Problem:
Wie finden wir geometrische Formen (Linien, Kreise) in verrauschten Bildern?

### Zwei Hauptansätze:

**1. Hough-Transform (1962)**
- Transformiert Bildraum → Parameterraum
- Nutzt "Voting" Mechanismus
- Gut bei mehreren Objekten
- Deterministisch

**2. RANSAC (1981)**
- RANdom SAmple Consensus
- Iteratives Sampling + Voting
- Robust gegen Ausreißer
- Probabilistisch

---

## 📦 Setup & Installation

In [ ]:
# Bibliotheken installieren
!pip install opencv-python-headless scikit-image -q

# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, feature
from skimage.transform import hough_line, hough_line_peaks, hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
import random
from IPython.display import display, HTML

# Plotting-Einstellungen
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Alle Bibliotheken geladen!")
print(f"OpenCV Version: {cv2.__version__}")
print(f"NumPy Version: {np.__version__}")

## 🎨 Teil 1: Testbilder erstellen

Wir erstellen synthetische Bilder mit Linien und Kreisen + Rauschen.

In [ ]:
def create_line_image(width=400, height=400, num_lines=3, noise_level=0.01):
    """
    Erstellt ein Bild mit zufälligen Linien und Rauschen
    """
    img = np.zeros((height, width), dtype=np.uint8)

    # Zufällige Linien zeichnen
    for _ in range(num_lines):
        x1, y1 = np.random.randint(0, width), np.random.randint(0, height)
        x2, y2 = np.random.randint(0, width), np.random.randint(0, height)
        cv2.line(img, (x1, y1), (x2, y2), 255, 2)

    # Rauschen hinzufügen
    noise = np.random.rand(height, width) < noise_level
    img[noise] = 255

    return img

def create_circle_image(width=400, height=400, num_circles=3, noise_level=0.01):
    """
    Erstellt ein Bild mit zufälligen Kreisen und Rauschen
    """
    img = np.zeros((height, width), dtype=np.uint8)

    # Zufällige Kreise zeichnen
    for _ in range(num_circles):
        center_x = np.random.randint(50, width-50)
        center_y = np.random.randint(50, height-50)
        radius = np.random.randint(20, 60)
        cv2.circle(img, (center_x, center_y), radius, 255, 2)

    # Rauschen hinzufügen
    noise = np.random.rand(height, width) < noise_level
    img[noise] = 255

    return img

# Testbilder erstellen
line_img = create_line_image(num_lines=4, noise_level=0.02)
circle_img = create_circle_image(num_circles=3, noise_level=0.02)

# Anzeigen
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(line_img, cmap='gray')
axes[0].set_title('Testbild: Linien mit Rauschen', fontsize=12)
axes[0].axis('off')

axes[1].imshow(circle_img, cmap='gray')
axes[1].set_title('Testbild: Kreise mit Rauschen', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("✅ Testbilder erstellt!")

---

# 📐 Teil 2: Hough-Transform

## Theorie: Wie funktioniert die Hough-Transform?

### Für Linien:

**Bildraum → Parameterraum (Hough-Raum)**

1. **Liniengleichung in Polarkoordinaten:**
   - `ρ = x·cos(θ) + y·sin(θ)`
   - `ρ` = Abstand vom Ursprung
   - `θ` = Winkel

2. **Voting Mechanismus:**
   - Jeder Kantenpixel "stimmt" für alle möglichen Linien durch diesen Punkt
   - Im Hough-Raum entsteht eine Sinuskurve pro Pixel
   - Schnittpunkte vieler Kurven = Starke Linie im Bild!

3. **Peak Detection:**
   - Finde lokale Maxima im Hough-Raum
   - Diese entsprechen Linien im Originalbild

### Für Kreise:

**Kreisgleichung:**
- `(x - a)² + (y - b)² = r²`
- Parameter: Zentrum (a, b) und Radius r
- 3D-Parameterraum (a, b, r)

---

## 2.1 Hough-Transform für Linien

In [ ]:
def hough_line_detection(image, num_peaks=5, threshold_ratio=0.5):
    """
    Erkennt Linien mit Hough-Transform
    """
    # Kantenerkennung (wichtig für Hough!)
    edges = canny(image, sigma=3)

    # Hough-Transform durchführen
    # theta: Winkelbereich (0 bis 180 Grad)
    tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)
    h, theta, d = hough_line(edges, theta=tested_angles)

    # Peaks finden (Linien)
    _, angles, dists = hough_line_peaks(h, theta, d, num_peaks=num_peaks,
                                         threshold=threshold_ratio * h.max())

    return edges, h, theta, d, angles, dists

def plot_hough_lines(image, edges, h, theta, d, angles, dists):
    """
    Visualisiert Hough-Transform für Linien
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # 1. Original mit Kanten
    axes[0].imshow(edges, cmap='gray')
    axes[0].set_title('1. Kantenerkennung (Canny)', fontsize=12, fontweight='bold')
    axes[0].axis('off')

    # 2. Hough-Raum (Akkumulator)
    axes[1].imshow(np.log(1 + h), cmap='hot', aspect='auto')
    axes[1].set_title('2. Hough-Raum (Log-Scale)', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Winkel θ (Grad)')
    axes[1].set_ylabel('Distanz ρ (Pixel)')

    # Peaks markieren
    for angle, dist in zip(angles, dists):
        row = np.argmin(np.abs(d - dist))
        col = np.argmin(np.abs(theta - angle))
        axes[1].plot(col, row, 'go', markersize=10, markerfacecolor='lime')

    # 3. Erkannte Linien auf Original
    axes[2].imshow(image, cmap='gray')
    axes[2].set_title('3. Erkannte Linien', fontsize=12, fontweight='bold')
    axes[2].axis('off')

    # Linien einzeichnen
    for angle, dist in zip(angles, dists):
        (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
        axes[2].axline((x0, y0), slope=np.tan(angle + np.pi/2),
                      color='red', linewidth=2, alpha=0.8)

    plt.tight_layout()
    plt.show()

    print(f"✅ {len(angles)} Linien erkannt!")
    print(f"   Winkel: {np.rad2deg(angles).round(1)}°")
    print(f"   Distanzen: {dists.round(1)} Pixel")

# Hough-Transform für Linien ausführen
print("🔍 Hough-Transform für Linien wird durchgeführt...\n")
edges, h, theta, d, angles, dists = hough_line_detection(line_img, num_peaks=6)
plot_hough_lines(line_img, edges, h, theta, d, angles, dists)

### 💡 Interpretation:

**Bild 1 (Kanten):**
- Canny-Kantendetektor findet Pixelpositionen
- Jeder weiße Pixel ist ein Kandidat für eine Linie

**Bild 2 (Hough-Raum):**
- X-Achse: Winkel θ (0-180°)
- Y-Achse: Distanz ρ vom Bildursprung
- Helle Bereiche = viele "Votes" = starke Linien
- Grüne Punkte = erkannte Peaks

**Bild 3 (Ergebnis):**
- Rote Linien = erkannte Linien
- Überlagert auf Originalbild

## 2.2 Hough-Transform für Kreise

In [ ]:
def hough_circle_detection(image, radius_range=(20, 60), num_peaks=3):
    """
    Erkennt Kreise mit Hough-Transform
    """
    # Kantenerkennung
    edges = canny(image, sigma=2.0)

    # Hough-Transform für Kreise
    # radius_range: Welche Radien sollen gesucht werden?
    hough_radii = np.arange(radius_range[0], radius_range[1], 2)
    hough_res = hough_circle(edges, hough_radii)

    # Peaks finden (Kreise)
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                                total_num_peaks=num_peaks)

    return edges, hough_res, hough_radii, accums, cx, cy, radii

def plot_hough_circles(image, edges, hough_res, hough_radii, accums, cx, cy, radii):
    """
    Visualisiert Hough-Transform für Kreise
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))

    # 1. Kanten
    axes[0, 0].imshow(edges, cmap='gray')
    axes[0, 0].set_title('1. Kantenerkennung', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')

    # 2. Hough-Raum (für mittleren Radius)
    mid_idx = len(hough_radii) // 2
    axes[0, 1].imshow(hough_res[mid_idx], cmap='hot')
    axes[0, 1].set_title(f'2. Hough-Raum (r={hough_radii[mid_idx]}px)',
                        fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')

    # 3. Akkumulator über alle Radien
    accum_all = np.sum(hough_res, axis=0)
    axes[1, 0].imshow(accum_all, cmap='hot')
    axes[1, 0].set_title('3. Akkumulierte Votes (alle Radien)',
                        fontsize=12, fontweight='bold')

    # Zentren markieren
    for center_x, center_y in zip(cx, cy):
        axes[1, 0].plot(center_x, center_y, 'go', markersize=12,
                       markerfacecolor='lime', markeredgewidth=2)
    axes[1, 0].axis('off')

    # 4. Erkannte Kreise auf Original
    axes[1, 1].imshow(image, cmap='gray')
    axes[1, 1].set_title('4. Erkannte Kreise', fontsize=12, fontweight='bold')

    # Kreise einzeichnen
    for center_x, center_y, radius in zip(cx, cy, radii):
        circle = plt.Circle((center_x, center_y), radius,
                           color='red', fill=False, linewidth=2)
        axes[1, 1].add_patch(circle)
        axes[1, 1].plot(center_x, center_y, 'r+', markersize=15, markeredgewidth=2)

    axes[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

    print(f"✅ {len(radii)} Kreise erkannt!")
    for i, (x, y, r, acc) in enumerate(zip(cx, cy, radii, accums), 1):
        print(f"   Kreis {i}: Zentrum=({x}, {y}), Radius={r}px, Votes={acc}")

# Hough-Transform für Kreise ausführen
print("🔍 Hough-Transform für Kreise wird durchgeführt...\n")
edges_c, hough_res, hough_radii, accums, cx, cy, radii = hough_circle_detection(
    circle_img, radius_range=(15, 65), num_peaks=3
)
plot_hough_circles(circle_img, edges_c, hough_res, hough_radii, accums, cx, cy, radii)

### 💡 Interpretation:

**Bild 1 (Kanten):**
- Kantenpixel sind Kandidaten für Kreisbögen

**Bild 2 (Hough-Raum für einen Radius):**
- 2D-Akkumulator für Kreiszentren bei festem Radius
- Helle Bereiche = potenzielle Kreiszentren

**Bild 3 (Akkumulierte Votes):**
- Summe über alle getesteten Radien
- Grüne Punkte = erkannte Zentren

**Bild 4 (Ergebnis):**
- Rote Kreise = erkannte Kreise
- Rotes Kreuz = Kreiszentrum

---

# 🎲 Teil 3: RANSAC

## Theorie: Wie funktioniert RANSAC?

**RANSAC = RANdom SAmple Consensus**

### Algorithmus:

```
Wiederhole N Iterationen:
    1. Wähle zufällig minimale Anzahl Punkte
       (Linie: 2 Punkte, Kreis: 3 Punkte)
    
    2. Fitte Modell an diese Punkte
       (Berechne Linien-/Kreisparameter)
    
    3. Zähle "Inliers" (Punkte nahe am Modell)
       (Distanz < threshold)
    
    4. Wenn mehr Inliers als bisher:
       Speichere dieses Modell

Ergebnis: Modell mit den meisten Inliers
```

### Vorteile:
- ✅ Sehr robust gegen Ausreißer (bis zu 50% Outliers)
- ✅ Findet globales Optimum (bei genug Iterationen)
- ✅ Einfach zu implementieren

### Nachteile:
- ❌ Probabilistisch (Ergebnis kann variieren)
- ❌ Findet nur ein Objekt pro Durchlauf
- ❌ Benötigt gute Parameter (Iterationen, Threshold)

---

## 3.1 RANSAC für Linien

In [ ]:
def ransac_line(points, iterations=1000, threshold=3.0, min_inliers=20):
    """
    RANSAC für Linienerkennung

    Parameters:
    -----------
    points : array (N, 2)
        Punktkoordinaten (x, y)
    iterations : int
        Anzahl RANSAC-Iterationen
    threshold : float
        Maximale Distanz für Inliers
    min_inliers : int
        Minimale Anzahl Inliers für gültige Linie
    """
    best_inliers = []
    best_model = None

    for i in range(iterations):
        # 1. Wähle 2 zufällige Punkte
        idx = np.random.choice(len(points), 2, replace=False)
        p1, p2 = points[idx]

        # 2. Berechne Linienparameter (ax + by + c = 0)
        # Linie durch zwei Punkte
        if p2[0] - p1[0] == 0:  # Vertikale Linie
            a, b, c = 1, 0, -p1[0]
        else:
            slope = (p2[1] - p1[1]) / (p2[0] - p1[0])
            # y - y1 = m(x - x1) → mx - y + (y1 - mx1) = 0
            a = slope
            b = -1
            c = p1[1] - slope * p1[0]

        # Normalisieren
        norm = np.sqrt(a**2 + b**2)
        a, b, c = a/norm, b/norm, c/norm

        # 3. Berechne Distanzen aller Punkte zur Linie
        distances = np.abs(a * points[:, 0] + b * points[:, 1] + c)

        # 4. Zähle Inliers
        inliers = np.where(distances < threshold)[0]

        # 5. Update bestes Modell
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_model = (a, b, c)

    if len(best_inliers) >= min_inliers:
        return best_model, best_inliers
    else:
        return None, []

def extract_edge_points(image, max_points=1000):
    """
    Extrahiert Kantenpunkte aus Bild
    """
    edges = canny(image, sigma=2.0)
    y, x = np.where(edges)
    points = np.column_stack([x, y])

    # Sample wenn zu viele Punkte
    if len(points) > max_points:
        idx = np.random.choice(len(points), max_points, replace=False)
        points = points[idx]

    return points, edges

def plot_ransac_lines(image, points, edges, models, inliers_list):
    """
    Visualisiert RANSAC-Linienerkennung
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # 1. Kantenpunkte
    axes[0].imshow(edges, cmap='gray')
    axes[0].scatter(points[:, 0], points[:, 1], c='red', s=1, alpha=0.5)
    axes[0].set_title(f'1. Kantenpunkte (N={len(points)})', fontsize=12, fontweight='bold')
    axes[0].axis('off')

    # 2. RANSAC Prozess (Inliers vs Outliers)
    axes[1].set_xlim(0, image.shape[1])
    axes[1].set_ylim(image.shape[0], 0)
    axes[1].set_aspect('equal')

    # Alle Punkte als Outliers
    axes[1].scatter(points[:, 0], points[:, 1], c='lightgray', s=5, alpha=0.5, label='Outliers')

    # Inliers farblich hervorheben
    colors = ['red', 'blue', 'green', 'orange']
    for i, (model, inliers) in enumerate(zip(models, inliers_list)):
        if len(inliers) > 0:
            inlier_points = points[inliers]
            axes[1].scatter(inlier_points[:, 0], inlier_points[:, 1],
                          c=colors[i % len(colors)], s=10,
                          label=f'Linie {i+1} ({len(inliers)} Inliers)')

    axes[1].set_title('2. RANSAC: Inliers vs Outliers', fontsize=12, fontweight='bold')
    axes[1].legend(loc='upper right')
    axes[1].axis('off')

    # 3. Erkannte Linien auf Original
    axes[2].imshow(image, cmap='gray')

    for i, (model, inliers) in enumerate(zip(models, inliers_list)):
        if model is not None:
            a, b, c = model
            # Zeichne Linie über gesamtes Bild
            x = np.array([0, image.shape[1]])
            y = -(a * x + c) / b if b != 0 else np.array([0, 0])
            axes[2].plot(x, y, color=colors[i % len(colors)],
                       linewidth=3, alpha=0.8, label=f'Linie {i+1}')

    axes[2].set_title('3. Erkannte Linien', fontsize=12, fontweight='bold')
    axes[2].legend(loc='upper right')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

# RANSAC für Linien ausführen
print("🎲 RANSAC für Linien wird durchgeführt...\n")

# Kantenpunkte extrahieren
points, edges = extract_edge_points(line_img, max_points=2000)
print(f"Extrahiert: {len(points)} Kantenpunkte")

# Mehrere Linien finden (iterativ)
models = []
inliers_list = []
remaining_points = points.copy()

for line_num in range(4):  # Versuche 4 Linien zu finden
    if len(remaining_points) < 50:  # Zu wenig Punkte übrig
        break

    model, inliers = ransac_line(remaining_points, iterations=1000,
                                  threshold=3.0, min_inliers=30)

    if model is not None:
        models.append(model)
        # Konvertiere lokale Inlier-Indizes zurück zu globalen
        global_inliers = np.where(np.isin(np.arange(len(points)),
                                          np.searchsorted(points[:, 0],
                                                         remaining_points[inliers, 0])))[0]
        inliers_list.append(inliers)

        # Entferne Inliers für nächste Iteration
        remaining_points = np.delete(remaining_points, inliers, axis=0)
        print(f"Linie {line_num + 1}: {len(inliers)} Inliers gefunden")
    else:
        break

print(f"\n✅ Insgesamt {len(models)} Linien erkannt!\n")

# Visualisierung
plot_ransac_lines(line_img, points, edges, models, inliers_list)

### 💡 Interpretation:

**Bild 1 (Kantenpunkte):**
- Rote Punkte = alle extrahierten Kantenpixel
- Diese sind Input für RANSAC

**Bild 2 (Inliers vs Outliers):**
- Grau = Outliers (gehören zu keiner erkannten Linie)
- Farbig = Inliers (gehören zu erkannter Linie)
- Verschiedene Farben = verschiedene Linien

**Bild 3 (Ergebnis):**
- Farbige Linien = RANSAC-Ergebnis
- Jede Farbe entspricht einer erkannten Linie

## 3.2 RANSAC für Kreise

In [ ]:
def ransac_circle(points, iterations=1000, threshold=3.0, min_inliers=30):
    """
    RANSAC für Kreiserkennung

    Parameters:
    -----------
    points : array (N, 2)
        Punktkoordinaten (x, y)
    iterations : int
        Anzahl RANSAC-Iterationen
    threshold : float
        Maximale Distanz für Inliers
    min_inliers : int
        Minimale Anzahl Inliers für gültigen Kreis
    """
    best_inliers = []
    best_model = None

    for i in range(iterations):
        # 1. Wähle 3 zufällige Punkte
        idx = np.random.choice(len(points), 3, replace=False)
        p1, p2, p3 = points[idx]

        # 2. Berechne Kreisparameter durch 3 Punkte
        # Kreisgleichung: (x-a)² + (y-b)² = r²
        # System von Gleichungen lösen

        try:
            # Matrix aufstellen
            A = np.array([
                [2*(p2[0] - p1[0]), 2*(p2[1] - p1[1])],
                [2*(p3[0] - p1[0]), 2*(p3[1] - p1[1])]
            ])

            b = np.array([
                p2[0]**2 + p2[1]**2 - p1[0]**2 - p1[1]**2,
                p3[0]**2 + p3[1]**2 - p1[0]**2 - p1[1]**2
            ])

            # Löse für Zentrum (a, b)
            center = np.linalg.solve(A, b)

            # Berechne Radius
            radius = np.sqrt((p1[0] - center[0])**2 + (p1[1] - center[1])**2)

            # Prüfe auf sinnvolle Werte
            if radius < 5 or radius > 200:  # Filter unrealistische Radien
                continue

        except (np.linalg.LinAlgError, ValueError):
            # Punkte sind kollinear oder singular
            continue

        # 3. Berechne Distanzen aller Punkte zum Kreis
        distances = np.sqrt((points[:, 0] - center[0])**2 +
                           (points[:, 1] - center[1])**2)
        distances = np.abs(distances - radius)  # Distanz zur Kreislinie

        # 4. Zähle Inliers
        inliers = np.where(distances < threshold)[0]

        # 5. Update bestes Modell
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_model = (center[0], center[1], radius)

    if len(best_inliers) >= min_inliers:
        return best_model, best_inliers
    else:
        return None, []

def plot_ransac_circles(image, points, edges, models, inliers_list):
    """
    Visualisiert RANSAC-Kreiserkennung
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # 1. Kantenpunkte
    axes[0].imshow(edges, cmap='gray')
    axes[0].scatter(points[:, 0], points[:, 1], c='red', s=1, alpha=0.5)
    axes[0].set_title(f'1. Kantenpunkte (N={len(points)})', fontsize=12, fontweight='bold')
    axes[0].axis('off')

    # 2. Inliers vs Outliers
    axes[1].set_xlim(0, image.shape[1])
    axes[1].set_ylim(image.shape[0], 0)
    axes[1].set_aspect('equal')

    # Alle Punkte als Outliers
    axes[1].scatter(points[:, 0], points[:, 1], c='lightgray', s=5, alpha=0.5, label='Outliers')

    # Inliers farblich hervorheben
    colors = ['red', 'blue', 'green', 'orange']
    for i, (model, inliers) in enumerate(zip(models, inliers_list)):
        if len(inliers) > 0:
            inlier_points = points[inliers]
            axes[1].scatter(inlier_points[:, 0], inlier_points[:, 1],
                          c=colors[i % len(colors)], s=10,
                          label=f'Kreis {i+1} ({len(inliers)} Inliers)')

    axes[1].set_title('2. RANSAC: Inliers vs Outliers', fontsize=12, fontweight='bold')
    axes[1].legend(loc='upper right')
    axes[1].axis('off')

    # 3. Erkannte Kreise auf Original
    axes[2].imshow(image, cmap='gray')

    for i, (model, inliers) in enumerate(zip(models, inliers_list)):
        if model is not None:
            cx, cy, r = model
            circle = plt.Circle((cx, cy), r, color=colors[i % len(colors)],
                              fill=False, linewidth=3, alpha=0.8,
                              label=f'Kreis {i+1}')
            axes[2].add_patch(circle)
            axes[2].plot(cx, cy, '+', color=colors[i % len(colors)],
                       markersize=15, markeredgewidth=3)

    axes[2].set_title('3. Erkannte Kreise', fontsize=12, fontweight='bold')
    axes[2].legend(loc='upper right')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

# RANSAC für Kreise ausführen
print("🎲 RANSAC für Kreise wird durchgeführt...\n")

# Kantenpunkte extrahieren
points_c, edges_c = extract_edge_points(circle_img, max_points=2000)
print(f"Extrahiert: {len(points_c)} Kantenpunkte")

# Mehrere Kreise finden (iterativ)
models_c = []
inliers_list_c = []
remaining_points_c = points_c.copy()

for circle_num in range(3):  # Versuche 3 Kreise zu finden
    if len(remaining_points_c) < 50:
        break

    model, inliers = ransac_circle(remaining_points_c, iterations=2000,
                                    threshold=5.0, min_inliers=40)

    if model is not None:
        models_c.append(model)
        inliers_list_c.append(inliers)

        # Entferne Inliers für nächste Iteration
        remaining_points_c = np.delete(remaining_points_c, inliers, axis=0)
        cx, cy, r = model
        print(f"Kreis {circle_num + 1}: Zentrum=({cx:.1f}, {cy:.1f}), Radius={r:.1f}px, {len(inliers)} Inliers")
    else:
        break

print(f"\n✅ Insgesamt {len(models_c)} Kreise erkannt!\n")

# Visualisierung
plot_ransac_circles(circle_img, points_c, edges_c, models_c, inliers_list_c)

---

# ⚖️ Teil 4: Vergleich - Hough vs. RANSAC

## Side-by-Side Vergleich

In [ ]:
# Vergleichstabelle erstellen
comparison_html = """
<style>
table {
    width: 100%;
    border-collapse: collapse;
    margin: 20px 0;
    font-size: 14px;
}
th, td {
    border: 1px solid #ddd;
    padding: 12px;
    text-align: left;
}
th {
    background-color: #4CAF50;
    color: white;
    font-weight: bold;
}
tr:nth-child(even) {
    background-color: #f2f2f2;
}
.pro { color: green; font-weight: bold; }
.con { color: red; font-weight: bold; }
</style>

<h2>📊 Hough-Transform vs. RANSAC</h2>

<table>
  <tr>
    <th>Kriterium</th>
    <th>Hough-Transform</th>
    <th>RANSAC</th>
  </tr>
  <tr>
    <td><b>Grundprinzip</b></td>
    <td>Transformation in Parameterraum + Voting</td>
    <td>Iteratives Random Sampling + Consensus</td>
  </tr>
  <tr>
    <td><b>Deterministisch?</b></td>
    <td><span class="pro">✓ Ja</span> - Immer gleiches Ergebnis</td>
    <td><span class="con">✗ Nein</span> - Probabilistisch, Ergebnis variiert</td>
  </tr>
  <tr>
    <td><b>Mehrere Objekte?</b></td>
    <td><span class="pro">✓ Ja</span> - Findet alle Objekte gleichzeitig</td>
    <td><span class="con">✗ Schwierig</span> - Findet ein Objekt pro Durchlauf</td>
  </tr>
  <tr>
    <td><b>Robustheit gegen Rauschen</b></td>
    <td><span class="con">Mittel</span> - Rauschen erzeugt Votes</td>
    <td><span class="pro">✓ Sehr hoch</span> - Bis zu 50% Outliers</td>
  </tr>
  <tr>
    <td><b>Rechenaufwand</b></td>
    <td><span class="con">Hoch</span> - Speicher für Akkumulator<br>(Linien: O(N·M), Kreise: O(N·M·K))</td>
    <td><span class="pro">Moderat</span> - Nur wenige Punkte pro Iteration<br>(O(I·N), I=Iterationen)</td>
  </tr>
  <tr>
    <td><b>Parameterwahl</b></td>
    <td>Akkumulator-Auflösung, Schwellwerte</td>
    <td>Iterationen, Distanz-Threshold, Min-Inliers</td>
  </tr>
  <tr>
    <td><b>Kreise mit unbekanntem Radius</b></td>
    <td><span class="con">✗ Schwierig</span> - 3D-Akkumulator (a,b,r) sehr groß</td>
    <td><span class="pro">✓ Gut</span> - Radius wird aus 3 Punkten berechnet</td>
  </tr>
  <tr>
    <td><b>Beste Anwendung</b></td>
    <td>• Mehrere ähnliche Objekte<br>• Bekannte Größenbereiche<br>• Deterministisches Ergebnis wichtig</td>
    <td>• Einzelnes Objekt<br>• Viele Ausreißer<br>• Unbekannte Parameter</td>
  </tr>
</table>

<h3>🎯 Wann welche Methode?</h3>
<ul>
  <li><b>Hough-Transform verwenden für:</b>
    <ul>
      <li>Spurerkennung im Straßenverkehr (mehrere Fahrbahnmarkierungen)</li>
      <li>Dokumentenanalyse (viele Linien/Tabellen)</li>
      <li>Münzerkennung (bekannte Radien)</li>
    </ul>
  </li>
  <li><b>RANSAC verwenden für:</b>
    <ul>
      <li>Ebenen-Fitting in 3D-Punktwolken (viel Rauschen)</li>
      <li>Kamera-Kalibrierung (Ausreißer-robust)</li>
      <li>Objekterkennung mit Feature-Matching (viele falsche Matches)</li>
    </ul>
  </li>
</ul>
"""

display(HTML(comparison_html))

## Praktische Performance-Analyse

In [ ]:
import time

# Vergleiche Laufzeiten
print("⏱️ Performance-Vergleich: Hough vs. RANSAC\n")
print("="*60)

# Hough Linien
start = time.time()
edges_h, h, theta, d, angles, dists = hough_line_detection(line_img, num_peaks=4)
hough_line_time = time.time() - start

# RANSAC Linien
points, edges = extract_edge_points(line_img, max_points=2000)
start = time.time()
model, inliers = ransac_line(points, iterations=1000, threshold=3.0)
ransac_line_time = time.time() - start

# Hough Kreise
start = time.time()
edges_c, hough_res, hough_radii, accums, cx, cy, radii = hough_circle_detection(
    circle_img, radius_range=(15, 65), num_peaks=3
)
hough_circle_time = time.time() - start

# RANSAC Kreise
points_c, edges_c = extract_edge_points(circle_img, max_points=2000)
start = time.time()
model_c, inliers_c = ransac_circle(points_c, iterations=2000, threshold=5.0)
ransac_circle_time = time.time() - start

# Ergebnisse
print("LINIEN-ERKENNUNG:")
print(f"  Hough-Transform:  {hough_line_time*1000:.1f} ms")
print(f"  RANSAC:           {ransac_line_time*1000:.1f} ms")
print(f"  Verhältnis:       {hough_line_time/ransac_line_time:.2f}x\n")

print("KREIS-ERKENNUNG:")
print(f"  Hough-Transform:  {hough_circle_time*1000:.1f} ms")
print(f"  RANSAC:           {ransac_circle_time*1000:.1f} ms")
print(f"  Verhältnis:       {hough_circle_time/ransac_circle_time:.2f}x\n")

print("="*60)
print("\n💡 Beobachtungen:")
print("   - Hough oft schneller bei Linien (einfacher Parameterraum)")
print("   - RANSAC oft schneller bei Kreisen (3D-Akkumulator vermieden)")
print("   - RANSAC-Zeit hängt stark von Anzahl Iterationen ab")
print("   - Hough-Zeit hängt von Auflösung des Akkumulators ab")

---

# 🎓 Teil 5: Zusammenfassung & Quiz

## Wichtige Konzepte

### Hough-Transform:
✅ **Bildraum → Parameterraum**
- Jeder Bildpunkt "votet" für Parameter
- Akkumulator sammelt Votes
- Peaks im Akkumulator = Objekte im Bild

✅ **Linien:** Polarkoordinaten (ρ, θ)
- ρ = Distanz vom Ursprung
- θ = Winkel

✅ **Kreise:** Kreisgleichung (a, b, r)
- (x-a)² + (y-b)² = r²
- 3D-Akkumulator notwendig

### RANSAC:
✅ **Iteratives Sampling**
- Minimale Anzahl Punkte wählen
- Modell fitten
- Inliers zählen
- Bestes Modell speichern

✅ **Robust gegen Ausreißer**
- Funktioniert bis ~50% Outliers
- Probabilistisches Verfahren

✅ **Iterationen berechnen:**
```
N = log(1-p) / log(1-w^s)
p = gewünschte Erfolgswahrscheinlichkeit (z.B. 0.99)
w = Anteil Inliers (z.B. 0.5 für 50%)
s = Anzahl Punkte für Modell (Linie: 2, Kreis: 3)
```

---

## 🧪 Experimentier-Aufgaben

### Aufgabe 1: Parameter-Tuning
Ändern Sie die Parameter und beobachten Sie die Auswirkungen:

**Für Hough:**
- Anzahl Peaks (num_peaks)
- Threshold-Ratio
- Radius-Range bei Kreisen

**Für RANSAC:**
- Anzahl Iterationen
- Distanz-Threshold
- Min-Inliers

### Aufgabe 2: Robustheit testen
Erstellen Sie Bilder mit unterschiedlichem Rauschen-Level:

In [ ]:
# Experiment: Verschiedene Rauschen-Level
noise_levels = [0.01, 0.05, 0.10]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for i, noise in enumerate(noise_levels):
    # Bild mit Rauschen erstellen
    noisy_img = create_line_image(num_lines=3, noise_level=noise)

    # Hough anwenden
    edges_h, h, theta, d, angles, dists = hough_line_detection(noisy_img, num_peaks=3)

    # Original mit Rauschen
    axes[0, i].imshow(noisy_img, cmap='gray')
    axes[0, i].set_title(f'Rauschen: {noise*100:.1f}%', fontsize=11)
    axes[0, i].axis('off')

    # Hough-Ergebnis
    axes[1, i].imshow(noisy_img, cmap='gray')
    for angle, dist in zip(angles, dists):
        (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
        axes[1, i].axline((x0, y0), slope=np.tan(angle + np.pi/2),
                         color='red', linewidth=2)
    axes[1, i].set_title(f'{len(angles)} Linien erkannt', fontsize=11)
    axes[1, i].axis('off')

plt.suptitle('Robustheit-Test: Hough-Transform bei verschiedenen Rauschen-Leveln',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Beobachten Sie:")
print("   - Bei welchem Rauschen-Level beginnt Hough zu versagen?")
print("   - Wiederholen Sie das Experiment mit RANSAC - was ist robuster?")

### Aufgabe 3: Echtes Bild laden

Laden Sie ein eigenes Bild und wenden Sie beide Methoden an!

In [ ]:
from google.colab import files

# Bild hochladen
print("📤 Laden Sie ein Bild mit Linien oder Kreisen hoch:")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

    print(f"✅ Bild '{filename}' geladen! Größe: {img.shape}")

    # Anzeigen
    plt.figure(figsize=(10, 6))
    plt.imshow(img, cmap='gray')
    plt.title('Ihr hochgeladenes Bild', fontsize=14)
    plt.axis('off')
    plt.show()

    print("\n💡 Jetzt können Sie Hough oder RANSAC auf dieses Bild anwenden!")
    print("   Kopieren Sie Code aus den obigen Zellen und passen Sie ihn an.")
else:
    print("⚠️ Kein Bild hochgeladen.")

---

## 📖 Weiterführende Themen

### Für Interessierte:

1. **Generalized Hough Transform**
   - Erkennung beliebiger Formen (nicht nur Linien/Kreise)
   - Template-basiert

2. **Progressive RANSAC**
   - Adaptiert Iterationen basierend auf bisherigen Ergebnissen
   - Effizienter als Standard-RANSAC

3. **MLESAC / MSAC**
   - Maximum Likelihood RANSAC
   - Bessere Modell-Selektion

4. **Kombinierte Ansätze**
   - Hough für grobe Schätzung
   - RANSAC für Verfeinerung

---

## 🎯 Lernziel-Check

Können Sie folgende Fragen beantworten?

☑️ Was bedeutet "Transformation in den Parameterraum" bei Hough?

☑️ Warum braucht man für Kreise einen 3D-Akkumulator?

☑️ Wie funktioniert das Voting bei RANSAC?

☑️ Wann ist Hough besser als RANSAC und umgekehrt?

☑️ Wie beeinflussen die Parameter die Ergebnisse?

---

**🎉 Herzlichen Glückwunsch! Sie haben Hough-Transform und RANSAC gemeistert!**